In [1]:
import pandas as pd
import numpy as np

In [87]:
df = pd.read_csv('../result/result_max_per_gene_single.csv')
df

,Unnamed: 0,dbeta,gene
0,cg13670057,0.359170,ABL1
1,cg10266490,0.398036,ACOT11
2,cg10976772,0.355652,ACOT7
3,cg18770350,0.375086,ACTN2
4,cg09499849,0.372613,ACVR1
...,...,...,...
370,cg04738965,0.404657,ZIC1
371,cg16768018,0.457541,ZIC4
372,cg20985450,0.402938,ZIC5
373,cg21183523,0.381336,ZNF521


In [88]:
train = pd.read_csv('../champ_result/all_beta_normalized.csv')

In [89]:
df_gene = train[train['Unnamed: 0'].isin(df['Unnamed: 0'])]
df_gene.reset_index(drop=True, inplace=True)
print(df_gene.shape)
df_gene.head()

(375, 891)


,Unnamed: 0,1,2,3,4,5,6,7,8,9,...,881,882,883,884,885,886,887,888,889,890
0,cg03520644,0.229817,0.229817,0.079642,0.079642,0.180363,0.180363,0.182330,0.182330,0.156993,...,0.787966,0.787966,0.717043,0.717043,0.732963,0.732963,0.653539,0.653539,0.324511,0.324511
1,cg05047401,0.291908,0.291908,0.030616,0.030616,0.219142,0.219142,0.228138,0.228138,0.164601,...,0.790363,0.790363,0.774816,0.774816,0.738141,0.738141,0.780185,0.780185,0.721786,0.721786
2,cg06355129,0.232184,0.232184,0.261523,0.261523,0.176360,0.176360,0.253273,0.253273,0.083640,...,0.913858,0.913858,0.757479,0.757479,0.796541,0.796541,0.798998,0.798998,0.744522,0.744522
3,cg07790615,0.029336,0.029336,0.031738,0.031738,0.052790,0.052790,0.075713,0.075713,0.052144,...,0.793958,0.793958,0.713349,0.713349,0.809180,0.809180,0.798897,0.798897,0.584788,0.584788
4,cg09106903,0.091505,0.091505,0.066134,0.066134,0.126135,0.126135,0.094016,0.094016,0.039025,...,0.726509,0.726509,0.795497,0.795497,0.632736,0.632736,0.779592,0.779592,0.643527,0.643527


In [90]:
train_normal = df_gene.iloc[:,1:94:2]
train_tumor = df_gene.iloc[:,94::2]

train_tumor.columns=range(0,train_tumor.shape[1])
train_normal.columns=range(0,train_normal.shape[1])

train_tumor = train_tumor.T
train_normal = train_normal.T 

In [91]:
train_normal_avg = train_normal.mean(skipna=True, axis=0)

In [92]:
tt_dbeta=(train_tumor).subtract(train_normal_avg, axis = 1) 
tn_dbeta=(train_normal).subtract(train_normal_avg, axis = 1)

In [93]:
train_sensitivity = pd.DataFrame()
train_specificity = pd.DataFrame()
for i in np.arange(0.01,1,0.01):
    cp = i.__round__(2)
    TP = (tt_dbeta.abs() > cp).sum()
    FN = (tt_dbeta.abs() < cp).sum()
    FP = (tn_dbeta.abs() > cp).sum()
    TN = (tn_dbeta.abs() < cp).sum()
    sensitivity = TP/(TP+FN)
    specificity = TN/(TN+FP)

    train_sensitivity = pd.concat([train_sensitivity, sensitivity], ignore_index=True, axis=1)
    train_specificity = pd.concat([train_specificity, specificity], ignore_index=True, axis=1)        

In [94]:
# Youden Index (J)
Approach_J = train_sensitivity + train_specificity - 1 
max_columns_J = Approach_J.idxmax(axis=1)
max_values_J = Approach_J.max(axis=1)

sen_J = train_sensitivity.melt().loc[max_columns_J.index].iloc[:,1]
spe_J = train_specificity.melt().loc[max_columns_J.index].iloc[:,1]

In [95]:
# Concordance Probability Method (CZ)
Approach_CZ = train_sensitivity * train_specificity
max_columns_CZ = Approach_CZ.idxmax(axis=1)
max_values_CZ = Approach_CZ.max(axis=1)

sen_CZ = train_sensitivity.melt().loc[max_columns_CZ.index].iloc[:,1]
spe_CZ = train_specificity.melt().loc[max_columns_CZ.index].iloc[:,1]

In [96]:
# Closest to (0,1) Criteria (ER)
Approach_ER = ((1-train_sensitivity)**2 + (1-train_specificity)**2)
Approach_ER = Approach_ER.apply(np.sqrt)
min_columns_ER = Approach_ER.idxmin(axis=1)
min_values_ER = Approach_ER.min(axis=1)

sen_ER = train_sensitivity.melt().loc[min_columns_ER.index].iloc[:,1]
spe_ER = train_specificity.melt().loc[min_columns_ER.index].iloc[:,1]

In [97]:
from numpy import trapz
train_AUC = trapz(y = train_sensitivity[:], x = train_specificity[:] - 1 ,axis=1)

In [98]:
# F1 score
Approach_F1 = 2 * train_sensitivity * train_specificity / (train_sensitivity + train_specificity)
max_columns_F1 = Approach_F1.idxmax(axis=1)
max_values_F1 = Approach_F1.max(axis=1)

sen_F1 = train_sensitivity.melt().loc[max_columns_F1.index].iloc[:,1]
spe_F1 = train_specificity.melt().loc[max_columns_F1.index].iloc[:,1]

In [99]:
result_df = pd.DataFrame({
        'J': max_values_J, 
        'J cut-point': (max_columns_J+1)/100, 
        'J sensitivity': sen_J, 
        'J specificity': spe_J, 
        'CZ': max_values_CZ,
        'CZ cut-point': (max_columns_CZ+1)/100,
        'CZ sensitivity': sen_CZ,
        'CZ specificity': spe_CZ,
        'ER': min_values_ER,
        'ER cut-point': (min_columns_ER+1)/100,
        'ER sensitivity': sen_ER,
        'ER specificity': spe_ER,
        'F1': max_values_F1,
        'F1 cut-point': (max_columns_F1+1)/100,
        'F1 sensitivity': sen_F1,
        'F1 specificity': spe_F1,
        'AUC': train_AUC
    })
result_df

,J,J cut-point,J sensitivity,J specificity,CZ,CZ cut-point,CZ sensitivity,CZ specificity,ER,ER cut-point,ER sensitivity,ER specificity,F1,F1 cut-point,F1 sensitivity,F1 specificity,AUC
0,0.872180,0.16,0.997494,0.106383,0.872180,0.16,0.997494,0.106383,0.124636,0.15,0.997494,0.106383,0.931727,0.16,0.997494,0.106383,0.854743
1,0.893510,0.25,1.000000,0.063830,0.895323,0.25,1.000000,0.063830,0.087829,0.25,1.000000,0.063830,0.945676,0.25,1.000000,0.063830,0.915827
2,0.840879,0.12,0.984962,0.106383,0.843812,0.12,0.984962,0.106383,0.131504,0.10,0.984962,0.106383,0.916749,0.12,0.984962,0.106383,0.838026
3,0.854637,0.12,0.979950,0.212766,0.854637,0.12,0.979950,0.212766,0.143299,0.06,0.979950,0.212766,0.921622,0.12,0.979950,0.212766,0.737135
4,0.764464,0.11,0.977444,0.106383,0.772676,0.11,0.977444,0.106383,0.197618,0.11,0.977444,0.106383,0.875820,0.11,0.977444,0.106383,0.793393
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370,0.819549,0.20,0.994987,0.042553,0.819549,0.20,0.994987,0.042553,0.161058,0.17,0.994987,0.042553,0.900826,0.20,0.994987,0.042553,0.890071
371,0.798272,0.14,0.982456,0.127660,0.802112,0.14,0.982456,0.127660,0.180526,0.13,0.982456,0.127660,0.892091,0.14,0.982456,0.127660,0.800005
372,0.869674,0.19,0.992481,0.127660,0.869674,0.19,0.992481,0.127660,0.114771,0.17,0.992481,0.127660,0.930727,0.17,0.992481,0.127660,0.826214
373,0.682984,0.22,0.972431,0.021277,0.689276,0.22,0.972431,0.021277,0.266849,0.11,0.972431,0.021277,0.819112,0.22,0.972431,0.021277,0.835200


In [100]:
result_df.to_csv('../result/result_max_per_gene_single_cp.csv', index=False)

In [101]:
test = pd.read_csv('../champ_result/all_beta_normalized_test.csv')

In [102]:
df_gene = test[test['Unnamed: 0'].isin(df['Unnamed: 0'])]
df_gene.reset_index(drop=True, inplace=True)
print(df_gene.shape)
df_gene.head()

(375, 901)


,Unnamed: 0,1,2,3,4,5,6,7,8,9,...,891,892,893,894,895,896,897,898,899,900
0,cg03520644,0.292910,0.292910,0.251303,0.251303,0.259736,0.259736,0.220520,0.220520,0.226304,...,0.620474,0.620474,0.587984,0.587984,0.725763,0.725763,0.543097,0.543097,0.627478,0.627478
1,cg05047401,0.504081,0.504081,0.562472,0.562472,0.537165,0.537165,0.482329,0.482329,0.297942,...,0.773014,0.773014,0.681243,0.681243,0.699969,0.699969,0.720280,0.720280,0.715946,0.715946
2,cg06355129,0.266061,0.266061,0.203968,0.203968,0.346129,0.346129,0.484779,0.484779,0.140203,...,0.222561,0.222561,0.729291,0.729291,0.815345,0.815345,0.728010,0.728010,0.208163,0.208163
3,cg07790615,0.101333,0.101333,0.053834,0.053834,0.111712,0.111712,0.213167,0.213167,0.044492,...,0.702896,0.702896,0.742537,0.742537,0.722355,0.722355,0.619104,0.619104,0.588905,0.588905
4,cg09106903,0.192771,0.192771,0.218550,0.218550,0.222275,0.222275,0.105394,0.105394,0.136350,...,0.701276,0.701276,0.531955,0.531955,0.759788,0.759788,0.750459,0.750459,0.119129,0.119129


In [103]:
test_normal = df_gene.iloc[:,2:102:2]
test_tumor = df_gene.iloc[:,102::2]

test_tumor.columns=range(0,test_tumor.shape[1])
test_normal.columns=range(0,test_normal.shape[1])

test_tumor = test_tumor.T
test_normal = test_normal.T 

In [104]:
# Computing IQR
def IQR(df):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1
    upper_fence=Q3+IQR*1.5
    lower_fence=Q1-IQR*1.5
    return upper_fence,lower_fence
def no_outlier(df):
    upper_fence,lower_fence=IQR(df)
    ddf=df[(df>lower_fence)&(df<upper_fence)]
    return ddf

In [105]:
test_tumor = no_outlier(test_tumor)
test_normal = no_outlier(test_normal)

In [106]:
test_normal_avg = test_normal.mean(skipna=True, axis=0)

In [107]:
ttt_dbeta=(test_tumor).subtract(test_normal_avg, axis = 1)
ttn_dbeta=(test_normal).subtract(test_normal_avg, axis = 1)

In [108]:
cp_list = ['J cut-point', 'CZ cut-point', 'ER cut-point', 'F1 cut-point']
df_cpc = result_df[cp_list]
test_cp_sensitivity = pd.DataFrame()
test_cp_specificity = pd.DataFrame()

for i in range(0, df_cpc.shape[1]):
    cp = df_cpc[cp_list[i]]
    TP = (ttt_dbeta.abs() > cp).sum()
    FN = (ttt_dbeta.abs() < cp).sum()
    FP = (ttn_dbeta.abs() > cp).sum()
    TN = (ttn_dbeta.abs() < cp).sum()
    sensitivity = TP/(TP+FN)
    specificity = TN/(TN+FP)
    test_cp_sensitivity = pd.concat([test_cp_sensitivity, sensitivity], ignore_index=True, axis=1)
    test_cp_specificity = pd.concat([test_cp_specificity, specificity], ignore_index=True, axis=1)
    
test_cp_sensitivity.columns = cp_list
test_cp_specificity.columns = cp_list

In [109]:
test_sensitivity = pd.DataFrame()
test_specificity = pd.DataFrame()
for i in np.arange(0.01,1,0.01):
    cp = i.__round__(2)
    TP = (ttt_dbeta.abs() > cp).sum()
    FN = (ttt_dbeta.abs() < cp).sum()
    FP = (ttn_dbeta.abs() > cp).sum()
    TN = (ttn_dbeta.abs() < cp).sum()
    sensitivity = TP/(TP+FN)
    specificity = TN/(TN+FP)

    test_sensitivity = pd.concat([test_sensitivity, sensitivity], ignore_index=True, axis=1)
    test_specificity = pd.concat([test_specificity, specificity], ignore_index=True, axis=1)  

In [110]:
from numpy import trapz
test_AUC = trapz(y = test_sensitivity[:], x = test_specificity[:], axis=1)

In [112]:
print(test_cp_sensitivity.shape)
test_cp_sensitivity.head(6)

(375, 4)


,J cut-point,CZ cut-point,ER cut-point,F1 cut-point
0,0.940415,0.940415,0.943005,0.940415
1,0.673469,0.673469,0.673469,0.673469
2,0.879093,0.879093,0.899244,0.879093
3,0.867500,0.867500,0.897500,0.867500
4,0.817500,0.817500,0.817500,0.817500
5,0.660000,0.660000,0.697500,0.660000


In [113]:
print(test_cp_specificity.shape)
test_cp_specificity.head(6)

(375, 4)


,J cut-point,CZ cut-point,ER cut-point,F1 cut-point
0,1.000000,1.000000,1.000000,1.000000
1,0.977273,0.977273,0.977273,0.977273
2,0.914894,0.914894,0.851064,0.914894
3,1.000000,1.000000,0.977778,1.000000
4,0.914894,0.914894,0.914894,0.914894
5,1.000000,1.000000,0.860465,1.000000


In [114]:
import numpy as np
def J_method(sen, spe):
    J = sen + spe - 1
    return J
def CZ_method(sen, spe):
    CZ = sen * spe 
    return CZ
def ER_method(sen, spe):
    ER = (1-sen)**2 + (1-spe)**2
    ER = ER.apply(np.sqrt)
    return ER
def UI_method(sen, spe, auc):
    UI = abs(sen.subtract(auc, axis=0)) + abs(spe.subtract(auc, axis=0))
    return UI
def F1_method(sen, spe):
    F1 = 2 * sen * spe / (sen + spe)
    return F1

In [115]:
J = J_method(test_cp_sensitivity, test_cp_specificity)
CZ = CZ_method(test_cp_sensitivity, test_cp_specificity)
ER = ER_method(test_cp_sensitivity, test_cp_specificity)
F1 = F1_method(test_cp_sensitivity, test_cp_specificity)

In [116]:
df_cross_validation = pd.concat([J, CZ, ER,F1],keys = ['J', 'CZ', 'ER','F1'], axis=1)
# merge two level of columns
df_cross_validation.columns = df_cross_validation.columns.map(','.join)
df_cross_validation.columns = df_cross_validation.columns.str.replace(' cut-point', '')
print(df_cross_validation.shape)
df_cross_validation.head()

(375, 16)


,"J,J","J,CZ","J,ER","J,F1","CZ,J","CZ,CZ","CZ,ER","CZ,F1","ER,J","ER,CZ","ER,ER","ER,F1","F1,J","F1,CZ","F1,ER","F1,F1"
0,0.940415,0.940415,0.943005,0.940415,0.940415,0.940415,0.943005,0.940415,0.059585,0.059585,0.056995,0.059585,0.969292,0.969292,0.970667,0.969292
1,0.650742,0.650742,0.650742,0.650742,0.658163,0.658163,0.658163,0.658163,0.327321,0.327321,0.327321,0.327321,0.797415,0.797415,0.797415,0.797415
2,0.793987,0.793987,0.750308,0.793987,0.804277,0.804277,0.765314,0.804277,0.147857,0.147857,0.179816,0.147857,0.896636,0.896636,0.874491,0.896636
3,0.867500,0.867500,0.875278,0.867500,0.867500,0.867500,0.877556,0.867500,0.132500,0.132500,0.104881,0.132500,0.929050,0.929050,0.935921,0.929050
4,0.732394,0.732394,0.732394,0.732394,0.747926,0.747926,0.747926,0.747926,0.201369,0.201369,0.201369,0.201369,0.863459,0.863459,0.863459,0.863459


In [117]:
method_list = ['J','CZ','ER','F1']
for i in method_list:
    ascending = False
    if i == 'ER':
        ascending = True
    for j in method_list:    
        df_cross_validation[f'{i},{j}_rank'] = df_cross_validation[f'{i},{j}'].rank(ascending=ascending)

In [118]:
df_cross_validation = pd.concat([df, df_cross_validation], axis=1)
df_cross_validation = pd.concat([df_cross_validation, pd.DataFrame(train_AUC, columns=['train_AUC'])], axis=1)
df_cross_validation = pd.concat([df_cross_validation, pd.DataFrame(test_AUC, columns=['test_AUC'])], axis=1)
df_cross_validation

,Unnamed: 0,dbeta,gene,"J,J","J,CZ","J,ER","J,F1","CZ,J","CZ,CZ","CZ,ER",...,"ER,J_rank","ER,CZ_rank","ER,ER_rank","ER,F1_rank","F1,J_rank","F1,CZ_rank","F1,ER_rank","F1,F1_rank",train_AUC,test_AUC
0,cg13670057,0.359170,ABL1,0.940415,0.940415,0.943005,0.940415,0.940415,0.940415,0.943005,...,19.0,19.0,16.0,18.0,15.0,15.0,10.0,15.0,0.854743,0.723671
1,cg10266490,0.398036,ACOT11,0.650742,0.650742,0.650742,0.650742,0.658163,0.658163,0.658163,...,348.0,346.0,338.0,347.0,340.0,339.0,326.0,335.0,0.915827,0.894162
2,cg10976772,0.355652,ACOT7,0.793987,0.793987,0.750308,0.793987,0.804277,0.804277,0.765314,...,129.0,129.0,184.0,131.0,163.0,163.0,206.0,163.0,0.838026,0.770084
3,cg18770350,0.375086,ACTN2,0.867500,0.867500,0.875278,0.867500,0.867500,0.867500,0.877556,...,104.0,106.0,63.0,107.0,83.0,83.0,55.0,81.0,0.737135,0.702250
4,cg09499849,0.372613,ACVR1,0.732394,0.732394,0.732394,0.732394,0.747926,0.747926,0.747926,...,220.0,222.0,218.0,220.0,247.0,250.0,228.0,245.0,0.793393,0.762793
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370,cg04738965,0.404657,ZIC1,0.814249,0.814249,0.839695,0.814249,0.814249,0.814249,0.839695,...,193.0,196.0,145.0,198.0,157.0,156.0,103.0,156.0,0.890071,0.843989
371,cg16768018,0.457541,ZIC4,0.810000,0.810000,0.820000,0.810000,0.810000,0.810000,0.820000,...,200.0,202.0,185.0,204.0,166.0,165.0,136.0,166.0,0.800005,0.811766
372,cg20985450,0.402938,ZIC5,0.804282,0.804282,0.699948,0.699948,0.813368,0.813368,0.714360,...,118.0,120.0,280.0,284.0,151.0,150.0,279.0,291.0,0.826214,0.925431
373,cg21183523,0.381336,ZNF521,0.684022,0.684022,0.654565,0.684022,0.695870,0.695870,0.682609,...,321.0,321.0,284.0,317.0,313.0,313.0,293.0,310.0,0.835200,0.872310


In [35]:
# calculate correlation between J,J	and J,CZ
J_cor = [
    df_cross_validation['J,J'].corr(df_cross_validation['J,J']),
    df_cross_validation['J,J'].corr(df_cross_validation['J,CZ']),
    df_cross_validation['J,J'].corr(df_cross_validation['J,ER']),
    df_cross_validation['J,J'].corr(df_cross_validation['J,F1'])]
CZ_cor = [
    df_cross_validation['J,CZ'].corr(df_cross_validation['J,J']),
    df_cross_validation['J,CZ'].corr(df_cross_validation['J,CZ']),
    df_cross_validation['J,CZ'].corr(df_cross_validation['J,ER']),
    df_cross_validation['J,CZ'].corr(df_cross_validation['J,F1'])]
ER_cor = [
    df_cross_validation['J,ER'].corr(df_cross_validation['J,J']),
    df_cross_validation['J,ER'].corr(df_cross_validation['J,CZ']),
    df_cross_validation['J,ER'].corr(df_cross_validation['J,ER']),
    df_cross_validation['J,ER'].corr(df_cross_validation['J,F1'])]
F1_cor = [
    df_cross_validation['J,F1'].corr(df_cross_validation['J,J']),
    df_cross_validation['J,F1'].corr(df_cross_validation['J,CZ']),
    df_cross_validation['J,F1'].corr(df_cross_validation['J,ER']),
    df_cross_validation['J,F1'].corr(df_cross_validation['J,F1'])]

 
J_cor += J_cor[:1]
CZ_cor += CZ_cor[:1]
ER_cor += ER_cor[:1]
F1_cor += F1_cor[:1]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Sample data for the five categories

categories = ['J', 'CZ', 'ER', 'F1']
num_categories = len(categories)
# Create an array of evenly spaced angles to represent each category
angles = np.linspace(0, 2 * np.pi, num_categories, endpoint=False).tolist()
angles += angles[:1]

# Create a figure with 2x3 subplots
fig, axes = plt.subplots(2, 3, figsize=(15, 10), subplot_kw=dict(polar=True))
fig.suptitle('Category Correlations', fontsize=16)

# Loop through the categories and create subplots
for i, (ax, category) in enumerate(zip(axes.flatten(), categories)):
    
    ax.fill(angles, eval(f'{category}_cor'), 'b', alpha=0.1)
    ax.plot(angles, eval(f'{category}_cor'), 'b', linewidth=2)
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(categories)
    ax.set_title(f'{category} method correlation')

# Adjust the layout
plt.tight_layout()
plt.subplots_adjust(top=0.85)
plt.show()


In [119]:
dmp = pd.read_csv('../champ_result/DMP_result_test.csv')
# remove gene columns
dmp = dmp.drop(dmp.columns[-10], axis=1)

In [120]:
merge = pd.merge(df_cross_validation, dmp, on='Unnamed: 0', how='left')

In [121]:
# drop row with nan in feature
merge.dropna(subset=['feature'], inplace=True)
merge

,Unnamed: 0,dbeta,gene,"J,J","J,CZ","J,ER","J,F1","CZ,J","CZ,CZ","CZ,ER",...,Type,feature,cgi,feat.cgi,UCSC_CpG_Islands_Name,DHS,Enhancer,Phantom,Probe_SNPs,Probe_SNPs_10
0,cg13670057,0.359170,ABL1,0.940415,0.940415,0.943005,0.940415,0.940415,0.940415,0.943005,...,II,Body,opensea,Body-opensea,NaN,NaN,True,NaN,NaN,NaN
1,cg10266490,0.398036,ACOT11,0.650742,0.650742,0.650742,0.650742,0.658163,0.658163,0.658163,...,II,TSS200,opensea,TSS200-opensea,NaN,NaN,NaN,NaN,NaN,NaN
2,cg10976772,0.355652,ACOT7,0.793987,0.793987,0.750308,0.793987,0.804277,0.804277,0.765314,...,II,Body,opensea,Body-opensea,NaN,True,True,NaN,NaN,NaN
3,cg18770350,0.375086,ACTN2,0.867500,0.867500,0.875278,0.867500,0.867500,0.867500,0.877556,...,I,1stExon,island,1stExon-island,chr1:236849472-236850323,True,True,high-CpG:234916417-234916759,NaN,NaN
4,cg09499849,0.372613,ACVR1,0.732394,0.732394,0.732394,0.732394,0.747926,0.747926,0.747926,...,II,5'UTR,opensea,5'UTR-opensea,NaN,NaN,True,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370,cg04738965,0.404657,ZIC1,0.814249,0.814249,0.839695,0.814249,0.814249,0.814249,0.839695,...,I,1stExon,island,1stExon-island,chr3:147126988-147128999,True,True,NaN,NaN,NaN
371,cg16768018,0.457541,ZIC4,0.810000,0.810000,0.820000,0.810000,0.810000,0.810000,0.820000,...,I,Body,island,Body-island,chr3:147108511-147111703,NaN,NaN,NaN,NaN,NaN
372,cg20985450,0.402938,ZIC5,0.804282,0.804282,0.699948,0.699948,0.813368,0.813368,0.714360,...,I,Body,island,Body-island,chr13:100620241-100624348,NaN,NaN,NaN,NaN,rs61080730
373,cg21183523,0.381336,ZNF521,0.684022,0.684022,0.654565,0.684022,0.695870,0.695870,0.682609,...,II,Body,shore,Body-shore,chr18:22930018-22930904,NaN,NaN,NaN,NaN,NaN


In [122]:
merge.to_csv('../result/result_test_cut_point_cross_vali_dmptest.csv', index=False)